In [1]:
import requests, json

In [2]:
# Use cached results from previous user searches (Skyscanner API on RapidAPI)

headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': "a25c140655mshcc3de02bc48e78fp1d8ae5jsn716b89dc608a"
    }

Start with one-way tickets and find cheapest option

In [59]:
# from Spain (ES-sky) - MAD (MAD-sky) & Switzerland (CH-sky) - Zurich (ZURI-sky)
origin_list = ["MAD-sky", "ZURI-sky"]

# where we are doing search from 
origin_country = "ES"

# list of possible destinations 
destination_list = ["LOND-sky", "PARI-sky", "ROME-sky"]

currency = "EUR"
locale = "en-US"
date = "2021-02-01" #yyyy-mm-dd
maxbudget = 200

#for origin in origin_list:
    #print(f'For origin {origin}:')
    #print()
    
    #for destination in destination_list:
        #myurl = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/" + originCountry + "/" + currency + "/" + locale + "/"  + destination + "/" + origin + "/"+ date
        #response = requests.request("GET", myurl, headers=headers)

        #print(f'For destination {destination}:')
        #print(response.text)
        #print()
        #print('--------')
        #print()
    
    #print('########')
    #print()

In [30]:
# match airport IDs to names of origin and destination
# add to options list

airports = {}
options = []

for origin in origin_list:
    for destination in destination_list:
        myurl = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/" + originCountry + "/" + currency + "/" + locale + "/"  + origin + "/" + destination + "/"+ date
        response = requests.request("GET", myurl, headers=headers)
        temp = json.loads(response.text)
        
        if("Quotes" in temp):
            for places in temp["Places"]:
                airports[places["PlaceId"]] = places["Name"] 
                
            for quotes in temp["Quotes"]:
                print("-----------")
                ori = quotes["OutboundLeg"]["OriginId"]
                dest = quotes["OutboundLeg"]["DestinationId"]
                price = quotes["MinPrice"]

                print(f"Journey:  {airports[ori]}  --> {airports[dest]}")
                print(f"Price: {price} EUR")
                
                options.append({"origin": airports[ori], "destination": airports[dest], "price": price})

-----------
Journey:  Madrid  --> London Stansted
Price: 31 EUR
-----------
Journey:  Madrid  --> London Gatwick
Price: 31 EUR
-----------
Journey:  Madrid  --> Paris Orly
Price: 36 EUR
-----------
Journey:  Madrid  --> Rome Ciampino
Price: 22 EUR
-----------
Journey:  Zurich  --> London Heathrow
Price: 64 EUR
-----------
Journey:  Zurich  --> London Heathrow
Price: 101 EUR
-----------
Journey:  Zurich  --> Paris Orly
Price: 74 EUR
-----------
Journey:  Zurich  --> Paris Charles de Gaulle
Price: 105 EUR
-----------
Journey:  Zurich  --> Rome Fiumicino
Price: 67 EUR
-----------
Journey:  Zurich  --> Rome Fiumicino
Price: 81 EUR


In [31]:
options

[{'origin': 'Madrid', 'destination': 'London Stansted', 'price': 31},
 {'origin': 'Madrid', 'destination': 'London Gatwick', 'price': 31},
 {'origin': 'Madrid', 'destination': 'Paris Orly', 'price': 36},
 {'origin': 'Madrid', 'destination': 'Rome Ciampino', 'price': 22},
 {'origin': 'Zurich', 'destination': 'London Heathrow', 'price': 64},
 {'origin': 'Zurich', 'destination': 'London Heathrow', 'price': 101},
 {'origin': 'Zurich', 'destination': 'Paris Orly', 'price': 74},
 {'origin': 'Zurich', 'destination': 'Paris Charles de Gaulle', 'price': 105},
 {'origin': 'Zurich', 'destination': 'Rome Fiumicino', 'price': 67},
 {'origin': 'Zurich', 'destination': 'Rome Fiumicino', 'price': 81}]

In [32]:
dd_options = [options, options]

In [55]:
# Find common destination and calculate total price

valid_destinations = []
price = []

for option_1 in dd_options[0]:
    for option_2 in dd_options[1]:
        
        if (option_1["origin"] != option_2["origin"]) & (option_1["destination"] == option_2["destination"]) :
            ori_1 = option_1["origin"]
            ori_2 = option_2["origin"]
            dest = option_1["destination"] 
            
            if dest not in valid_destinations:
                sum_price = option_1["price"] + option_2["price"]
                valid_destinations.append(dest)
                price.append(sum_price)

                print(f"Common destination:")
                print(f"Journey: {ori_1} -> {dest}")
                print(f"Journey: {ori_2} -> {dest}")
                print(f"Total price: {sum_price}")
                print("-----------")


Common destination:
Journey: Madrid -> Paris Orly
Journey: Zurich -> Paris Orly
Total price: 110
-----------


In [58]:
# Find cheapest destination

cheapest = valid_destinations[price.index(min(price))]
print(f"Cheapest destination:")
print(f"Journey: {origin_list[0]} -> {cheapest}")
print(f"Journey: {origin_list[1]} -> {cheapest}")
print(f"Total price: {min(price)}")

Cheapest destination:
Journey: MAD-sky -> Paris Orly
Journey: ZURI-sky -> Paris Orly
Total price: 110
